<a href="https://colab.research.google.com/github/NizarMohd/HTML/blob/master/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Preparation**



**Check Python version.**

In [1]:
!python -V

Python 3.7.12


**Check Cuda version.**

In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


**Mount the Google drive.**

Mount your Google drive to store the dataset and the trained models.
Execute the cell below. Visit this [URL](https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code) to retrieve the authorization code and  enter the code at the prompt. 

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


**Create a project directory**

In [4]:
from os import path, chdir, getcwd, mkdir

# Choose a project name
projectName = "RunOrWalk"

# Project directory is in My Drive
projectDirectory = "/content/drive/My Drive/" + projectName

# Checks if cwd is in content folder
if getcwd() == "/content":
  # Makes project directory if it does not exist
  if not path.isdir(projectDirectory):
    mkdir(projectDirectory)
    print(f"Project {projectName} has been created!")
  else:
    print(f"Project {projectName} already exist!")
  # Changes to project directory
  chdir(projectDirectory)

print(f"The current working directory is {getcwd()}")

Project RunOrWalk already exist!
The current working directory is /content/drive/My Drive/RunOrWalk


**Install dependencies.**

In [5]:
pip install torch torchvision

In [6]:
pip install opencv-python

**Check GPU usage.**

In [7]:
# Check if runtime uses GPU
import torch

gpu_name = torch.cuda.get_device_name(0)
print("Using GPU", gpu_name)

Using GPU Tesla P100-PCIE-16GB


Load Dataset

In [60]:

# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.utils.data import Dataset
from torch.utils.data import Sampler
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from torchvision import transforms
from torchvision.transforms import *


datasets = pd.read_csv('dataset.csv')

train_set, test_set = train_test_split(datasets, test_size = 0.1)

# # extract labels

# def extract_labels(df): 
#     x_train = [] 
#     y_train = [] 
 
#     i = 0 
#     while i + 1 < len(df): 
#         x_train_point = df[i: i + 1].filter(items=['acceleration_x', 'acceleration_y', 'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z']) 
#         x_train.append(x_train_point.to_numpy()) 
#         y_train.append(df['activity'].iloc[i]) 
#         i = i + 1 
 
#     x_train = torch.from_numpy(np.array(x_train))
#     y_train = torch.from_numpy(np.array(y_train))
     
#     return x_train, y_train

# train_x, train_y = extract_labels(train)
# test_x, test_y = extract_labels(test)

# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)



class MyDataset(Dataset):
    
    def __init__(self, is_train=True):
      if is_train:
        self.data = train_set
      else:
        self.data = test_set
      print(f'loaded data with dataset size {len(self)}')
 
      
    def __getitem__(self, idx):
        # print(self.data.iloc[idx])
        # sys.stdout.flush()
        _, _, _, _, activity,ax, ay,az, gx,gy,gz = self.data.iloc[idx] 
        image = [[0, 0, 0, 0, 0, 0],[ax,ay,az,gx,gy,gz], [0, 0, 0, 0, 0, 0]]
        image = torch.from_numpy(np.array(image))
        label = torch.tensor(activity)
        return image, label

    def __len__(self):
        return len(self.data)


CNN Model

In [61]:
from torch.nn import Conv1d, BatchNorm1d, MaxPool1d
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv1d(3, 64, kernel_size=3, stride=1, padding="same"),
            BatchNorm1d(64),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv1d(64, 196, kernel_size=3, stride=1, padding="same"),
            BatchNorm1d(196),
            ReLU(inplace=True),
            MaxPool1d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


Train Model

In [67]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import multiprocessing as mp
from torch.optim.lr_scheduler import ReduceLROnPlateau

#empty list to store train losses
train_losses = []
# empty list to store validation losses
val_losses = []


class AverageMeter(object):
    """
    A utility class to compute statisitcs of losses and accuracies
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy_score(outputs, targets):
    """
    Compute top k accuracy
    """
    batch_size = targets.size(0)

    _, pred = outputs.topk(1, 1, True)
    pred = pred.t()
    correct = pred.eq(targets.view(1, -1))
    n_correct_elems = correct.type(torch.FloatTensor).sum().item()

    return n_correct_elems / batch_size

def train():
    batch_size = 64
    num_epochs = 40
    num_workers = 2

    model = Net()
    # model = model.cuda().float()
    model = nn.DataParallel(model.cuda().double())

    train_set = MyDataset(is_train=True)
    validation_set = MyDataset(is_train=False)

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=True)
    validation_loader = torch.utils.data.DataLoader(
        validation_set, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=False)

    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.00000, momentum=0.8)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay = 0.0000)
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1, factor=0.3, verbose=True)


    # training loop
    for epoch in range(num_epochs):
        start_time = time.time()
        train_loss, val_loss = AverageMeter(), AverageMeter()
        train_acc, val_acc = AverageMeter(), AverageMeter()


        # train loop
        model.train()
        for i, (image, label) in enumerate(train_loader):
            image, label = image.cuda(),label.cuda()
            prediction = model(image)

            loss = criterion(prediction, label)
            train_loss.update(loss.item())
            
            acc = accuracy_score(prediction, label)
            train_acc.update(acc)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i % 100 == 99:
                print(f'training: iteration {i} / {len(train_loader)}, avg train loss = {train_loss.avg:.4f}, '
                      f'train accuracy {train_acc.avg:.4f}')

        # validation
        model.eval()
        for i, (image, label) in enumerate(validation_loader):
            image,label = image.cuda(),label.cuda()           
            with torch.no_grad():
                prediction = model(image)

                loss = criterion(prediction, label)
                val_loss.update(loss.item())

                acc = accuracy_score(prediction, label)
                val_acc.update(acc)

            if i % 100 == 99:
                print(f'validation: iteration {i} / {len(validation_loader)}, avg val loss = {val_loss.avg:.4f}, '
                      f'val accuracy {val_acc.avg:.4f}')

        # epoch summary
        print(f'Epoch {epoch}, train error {train_loss.avg:.4f}, val error {val_loss.avg:.4f}. '
              f'Train acc = {train_acc.avg:.4f}, val acc = {val_acc.avg:.4f}. '
              f'Time cost {(time.time() - start_time) / 60:.2f} min.\n')

        # lr scheduler
        scheduler.step(val_loss.avg)

        # checkpoint
        if epoch % 2 == 1:
            torch.save(model.state_dict(), f'ckpt_e{epoch}.pth')
    torch.save(model.state_dict(), 'A0183398M-Model.pth')
  
if __name__ == "__main__":
    train()


loaded data with dataset size 79729
loaded data with dataset size 8859
training: iteration 99 / 1246, avg train loss = 0.1914, train accuracy 0.9230
training: iteration 199 / 1246, avg train loss = 0.1456, train accuracy 0.9432
training: iteration 299 / 1246, avg train loss = 0.1225, train accuracy 0.9532
training: iteration 399 / 1246, avg train loss = 0.1084, train accuracy 0.9593
training: iteration 499 / 1246, avg train loss = 0.1002, train accuracy 0.9621
training: iteration 599 / 1246, avg train loss = 0.0948, train accuracy 0.9647
training: iteration 699 / 1246, avg train loss = 0.0915, train accuracy 0.9663
training: iteration 799 / 1246, avg train loss = 0.0857, train accuracy 0.9685
training: iteration 899 / 1246, avg train loss = 0.0833, train accuracy 0.9692
training: iteration 999 / 1246, avg train loss = 0.0807, train accuracy 0.9704
training: iteration 1099 / 1246, avg train loss = 0.0776, train accuracy 0.9716
training: iteration 1199 / 1246, avg train loss = 0.0757, tr

Test Accuracy

In [69]:
import torch
from torch import nn
# Change [your_python_file] to the actual file containing the definitons of the classes


def test():
    my_model = Net()
    my_model = nn.DataParallel(my_model.cuda().double())
    my_model.eval()
    batch_size = 64

    my_model.load_state_dict(torch.load('A0183398M-Model.pth'))
    test_set = MyDataset(is_train=False)


    # mp.set_start_method('spawn', force=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                      shuffle=False)

    # test accuracy
    test_acc = AverageMeter()
    for i, (image,label) in enumerate(test_loader):
        image, label = image.cuda(), label.cuda()
        with torch.no_grad():
            prediction = my_model(image)
            acc = accuracy_score(prediction, label)
            test_acc.update(acc)

        if i % 10 == 9:
            print(f'test: iteration {i} / {len(test_loader)}, '
                  f'test accuracy {test_acc.avg:.4f}')

    print(f'evaluation finished, val acc {test_acc.avg:.4f}')

if __name__ == "__main__":
    test()

loaded data with dataset size 8859
test: iteration 9 / 139, test accuracy 0.9938
test: iteration 19 / 139, test accuracy 0.9938
test: iteration 29 / 139, test accuracy 0.9906
test: iteration 39 / 139, test accuracy 0.9902
test: iteration 49 / 139, test accuracy 0.9894
test: iteration 59 / 139, test accuracy 0.9904
test: iteration 69 / 139, test accuracy 0.9906
test: iteration 79 / 139, test accuracy 0.9910
test: iteration 89 / 139, test accuracy 0.9906
test: iteration 99 / 139, test accuracy 0.9903
test: iteration 109 / 139, test accuracy 0.9903
test: iteration 119 / 139, test accuracy 0.9909
test: iteration 129 / 139, test accuracy 0.9912
evaluation finished, val acc 0.9916


In [ ]:
!pip install -q hls4ml

In [ ]:
import hls4ml

In [ ]:
config = hls4ml.utils.config_from_pytorch_model(model)